# XAI : LIME

## Overall Summary

- The paper addresses the problem of model interpretability in machine learning.
- The authors propose a local interpretable model-agnostic explanation technique (LIME) for explaining the predictions of any classifier.
- LIME generates a set of perturbed instances that are close to the original instance, but with some features randomly perturbed. The classifier's predictions for these perturbed instances are then used to train a simple, interpretable model that approximates the behavior of the classifier in the local region.
- The resulting model can be used to explain the original classifier's prediction by highlighting the important features that contributed to the prediction.
- The authors demonstrate the effectiveness of LIME on several datasets and classifiers, including text classifiers, image classifiers, and a deep neural network for image captioning.
- LIME can accurately identify the important features in each case, and the resulting explanations are both interpretable and intuitive.
- LIME can help to build trust in machine learning models by providing transparent and interpretable explanations of their predictions.
- The paper has had a significant impact on the field of machine learning, and LIME has become a widely used technique for model interpretation and explanation.

## Introduction

### XAI
설명가능한 인공지능(Explainable AI, XAI)란 무엇일까요?

Deep learning에서는 모델이 데이터와 결과 간의 상관관계를 추론하기 때문에, 결과 도출 과정이 사람이 이해하기 어려운 "블랙박스" 형태로 나타납니다.

따라서, 좋은 성능은 물론이고, 모델이 판단한 근거가 사람이 이해할 수 있고 전체 모집단에도 유효한 가정으로 검증되어야 합니다.

이를 위해, Deep learning 모델의 추론 근거 및 과정을 사람이 이해할 수 있도록 설명하는 방법론을 XAI라고 합니다.

XAI는 어려워 보일 수 있지만, 간단한 예로는 입력 영상에서 어느 부분을 중점적으로 보았는지 간단하게 표현하는 CAM(Class Activation Map) 방식도 있습니다.

### LIME

LIME은, 설명가능한 인공지능 분야에서 가장 널리 사용되는 방법 중 하나입니다. 
LIME은 어떤 분류기(classifier)의 예측을 설명하기 위해 근처의 지역(local region)에서 분류기의 동작을 근사하는 간단하고 해석 가능한 모델을 생성합니다. 
이를 위해, 원본 인스턴스와 유사하지만 일부 특징이 무작위로 변환된 변형 인스턴스 세트를 생성합니다. 
이 변형된 인스턴스들의 분류기 예측을 사용하여 해석 가능한 모델을 학습합니다. 
결과 모델은 분류기의 예측에 기여한 중요한 특징을 강조하여 분류기의 예측을 설명할 수 있습니다.

LIME은 텍스트 분류기, 이미지 분류기, 그리고 이미지 캡션 생성을 위한 딥 뉴럴 네트워크 등 여러 데이터셋과 분류기에서 효과적으로 작동하는 것으로 입증되었습니다. 

-> LIME은 모델을 설명하고 해석하는 데 매우 유용한 도구





## Abstract

논문 제목에서 알 수 있듯이 Classifier를 신뢰하기 위해서 XAI가 필요하다.

즉, XAI(Xplanaition Artificial Intelligence)를 통해 사람의 의사결정을 도와줄 수 있다는 발표

LIME 에서 제시하는 설명을 위해 필요한 특성

1. interpretable; 입력과 그 결과를 “사용자가 이해할 수 있는” 설명을 제공해야 함
2. local fidelity; 유사한 데이터에 대해서는 유사한 설명이 이뤄져야 함
3. model-agnostic; 어떤 형태의 모델에도 적용할 수 있어야 함
4. global perspective; 모델에 대한 전반적인 설명이 필요함

- (1), (2)를 설명에서 필수적인 특성으로 꼽고, 이를 기반으로 방법론을 구성함
- LIME은 두 가지 방식으로 설명: 
        첫 번째는 입력 attribute의 중요도(importance)를 찾는 방식, 
        두 번째는 중요한 attribute set을 선택하는 방식임
    두 가지 방식 각각 LIME, SP-LIME(Submodular Pick)으로 표현함




## Method of Lime

해석가능한 데이터 표현
 : 모델을 설명하기 앞서, 특성(features)과 해석 가능한 표현(interpretable data representations)의 차이를 구별
        - 해석가능한 설명은 보통 모델에 쓰이는 feature와는 관계없이 사람이 이해할 만한 표현을 사용해야 합니다. 
        - 예를 들면, 텍스트 모델의 경우 모델은 word embeddings과 같은 복잡한 feature를 사용하지만 인간에게는 단어의 존재 또는 부재와 같은 간단한 이진 벡터가 제공될 수 있습니다. 
        - 이미지 모델에서도 모델은 픽셀 당 RGB 값을 이용한 tensor를 이용하는 반면, 우리는 이미지의 특정 부분(슈퍼 픽셀)과 같은 요소의 존재 또는 부재를 제공받을 수 있습니다.

### 식 정의
- $x′∈R^d$  : 설명되는 데이터에 대한 기존의 표현
- $x′∈{0,1}^d$ ′  : 해석가능한 표현을 위한 이진 벡터
- f(x) : 평가하려고 하는 모델 f가 x를 넣었을 때 원하는 label에 대한 확률 값
- $g∈G$ :
    G : 해석가능한 모델들의 클래스, ex: linear model,decision tree..
    g : 사용자에게 visual or textual 요소를 제공하는 설명 모델. d ′ dimension의 {0,1} 벡터를 정의역으로 하며, 어떤 ‘해석 가능한 요소들’에 대한 존재 또는 부재로 표현됩니다.

모든 설명 모델 g가 해석가능 할 만큼 간단하지는 않기 때문에, Ω(g)를 설명 모델 g의 복잡성 측정(measure of complexity)라 하고, 해석가능성(interpretability)와 반대되는 개념으로 정의합니다. 
예를 들어, 결정 트리에서 트리의 깊이나, 선형 모델에서 가중치의 개수가 될 수 있습니다.

 설명의 대상이 되는 모델을 $f:R^d →R$이라 합시다. 즉, 분류에서는 f(x) : '특정 class에 속할 확률' 이 됩니다. 
            또한, 예시(instance) z와 x사이의 유사성 척도로 π_x(z)를 사용합니다. 
            결론적으로, L(f,g,π_x )를 π_x 로 정의되는 지역성(locality) 하에서 ‘g가 f로 근사하는 데 있어서 얼마나 믿음직스럽지 못한가’를 측정하는 척도로 사용할 수 있습니다.

우리는 해석가능성(interpretability)과 지역적 신뢰도(local fidelity) 두 가지를 보장하기 위해 복잡성인 Ω(g)를 인간이 해석가능 할 정도로 최대한 낮추면서, 
L(f,g,π_x)를 최소화 해야 합니다.
결국, LIME이 생성하는 설명은 아래와 같은 식으로 얻어집니다.

## method of Lime


논문 Figure 3그림에서 하늘색과 분홍색 배경은 deep learning인 모델 
의 복잡한 결정경계를 표현합니다. 
굵은 빨간색의 + 샘플에 대해서 설명하려고 할 때, 국소적인 지역(Locally)에서만 설명할 수 있는 interpretable한 모델 g를 근사시켜서 설명하는 방식입니다. 샘플을 중심으로 주변 데이터를 랜덤하게 생성하고, 그 샘플에 대해서 원 모델 f와 같은 결과를 내도록 학습시키면 샘플 주변에서는 모델의 추론 방식과 같게 학습될 것입니다.

추가적으로 생성하는 interpretable한 모델의 복잡도 (Complexity)를 
로 정의하고 loss에 포함시켜서, 사용자가 이해할 수 있는 설명을 만들도록 제한을 두었습니다.

tree의 경우 tree의 깊이, linear model의 경우 non-zero weight의 수 등…

또한 원래 모델과 유사하게 학습시키기 위해서 생성한 랜덤 데이터들의 가중치 
를 거리에 따라 다르게 두어 최대한 원 모델과 비슷하게 학습시킬 수 있게 하였습니다.

설명하고자 하는 샘플과 유사한 데이터들을 생성하기 위해서 원본 데이터에 각 attribute를 포함할지 제외할지를 1 또는 0으로 표현한 binary mask인 $z′∈{0,1}^d$를 생성합니다. 
이렇게 생성한 z'를 기반으로 실제 데이터 차원에서 샘플을 생성한 것을 $z′∈R^d$ 로 표현합니다. 
즉, 원 모델과 비슷한 결과를 내도록 모델 g를 근사시킵니다.

이 때 가중치는 π_x(z)로 정의되고,  D는 task에 따라 정의되는 거리함수를 의미합니다.



## Method of SP-LIME

Submodular pick은 설명에 요구되는 특성 중 4) global perspective를 표현하기 위한 방식입니다. 입력 값 중에서 어떤 set이 결정에 중요한지를 표현해서, 특히 vision 데이터에 대해서 한 눈에 알아볼 수 있도록 설명할 수 있다는 장점이 있습니다.

총 n개의 샘플 X=x_1+...+x_n에 대해서 학습시킨 각각의 g_i함수들의 weight를 바탕으로 최종적인 global importance를 생성합니다. weight를 matrix로 표현한 것을 explanation matrix  W_ij로 표현하는데, i는 각 샘플 j는 각 attribute를 의미합니다. 

explanation matrix를 기반으로, 각 attribute의 중요도 $I_j=\sqrt{\sum W_ij}$입니다. 
(논문에서는 feature importance로 표현합니다.)

행(row)마다 다른 샘플으로 각각 2개씩의 attribute를 중요하다고 판단하였습니다. 이 때 열(column)기준으로 보면 f2인 점선으로 표현된 부분이 가장 빈번하게 중요하다고 판단됨을 알 수 있습니다. feature importance가 높은 순으로 총 B 개의 attribute를 선택하여 set을 구성합니다.



## Results

XAI 방법은 그 성능을 증명하기가 어렵다는 점에서 한계가 있습니다. 
가장 중요한 특성이 “user의 수준에 맞는 설명”이기 때문에 설명을 잘했는지 못했는지를 객관적으로 입증하기가 어렵습니다. 
대부분 정성적인 방식으로 Amazon Mechanical Turk (AMT)를 통해 우수성을 입증합니다. 
이를 제외하고, 이 논문에서 흥미로웠던 실험은 객관적인 결과를 보여주기 위해 중요한 attribute들로 학습시킨 성능을 report했다는 점입니다. 
LIME의 결과로 중요도가 높은 attribute를 최대 10개 이용하여 모델을 학습시킨 후 그 성능을 비교하였습니다.

Linear Regression, Tree 모델, 그리고 두 데이터셋에서 모두 LIME의 recall이 가장 높은 것을 확인할 수 있습니다.